In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import torch

In [ ]:

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [13]:

#model.save_pretrained(MODEL)
text = "I love you, but I hate you."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


1) negative 0.7203
2) neutral 0.2415
3) positive 0.0383


In [ ]:
def analyse(model,text):
    #model.save_pretrained(MODEL)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [8]:
# load the json file
import json
with open('interstellar.json') as json_file:
    data = json.load(json_file)

In [29]:
# iterate the data with a window based of a time frame
# for each window, analyse the sentiment of the subtitles

def rolling_window_seconds(data, window_size):
    window_texts = []  # List to store the texts within the rolling window
    num_samples = len(data)

    for i in range(num_samples):
        sample_texts = []  # List to store the texts within each sample
        current_time = data[i][0]  # Initialize current_time with the start_time of the current subtitle

        for j in range(i, num_samples):
            start_time = data[j][0]
            end_time = data[j][1]

            if start_time <= current_time < end_time:  # The current_time falls within the current subtitle's timeframe
                print(data[j][2])
                sample_texts.append(data[j][2])  # Append the text to the sample_texts list

            if current_time + window_size <= end_time:  # Check whether the window_size fits within the current subtitle's timeframe
                current_time += window_size
            else:
                break  # The window_size doesn't fit within the current subtitle's timeframe, so break the inner loop

        window_texts.append(sample_texts)  # Append the sample_texts to the window_texts list

    return window_texts


In [33]:
import pandas as pd

def rolling_window(data, window_size):
    # Create a DataFrame from the data
    df = pd.DataFrame(data, columns=['time', 'irrelevant', 'text'])
    df['time'] = pd.to_timedelta(df['time'], unit='s')  # Convert to timedelta if 'time' is in seconds

    # Sort by time just in case the data is not in chronological order
    df.sort_values('time', inplace=True)

    # Create rolling window
    df.set_index('time', inplace=True)
    df['window_text'] = df['text'].rolling(window=f'{window_size}s').apply(lambda x: ' '.join(x))

    return df


In [41]:
window = 10
subs = []
for sub in data:
  subs.append(sub)
  start = sub[0]
  while start - subs[0][0] > window:
    print("Analyze", subs[:-1])
    subs.pop(0)


Analyze [[6.0, 12.074, 'Advertise your product or brand here\ncontact www.OpenSubtitles.org today']]
Analyze [[25.657, 28.203, 'My dad was a farmer.'], [29.098, 32.27, 'Like everybody else back then.'], [33.926, 36.125, "Of course he didn't start that way."]]
Analyze [[29.098, 32.27, 'Like everybody else back then.'], [33.926, 36.125, "Of course he didn't start that way."]]
Analyze [[33.926, 36.125, "Of course he didn't start that way."], [41.453, 43.607, "Your computer says you're too tight."], [43.716, 46.568, 'No, I got this!\n- Crossing the Straits.']]
Analyze [[41.453, 43.607, "Your computer says you're too tight."], [43.716, 46.568, 'No, I got this!\n- Crossing the Straits.'], [49.345, 51.226, "I've got to shut it down Cooper.\n- Heyyy."]]
Analyze [[43.716, 46.568, 'No, I got this!\n- Crossing the Straits.'], [49.345, 51.226, "I've got to shut it down Cooper.\n- Heyyy."], [52.543, 55.279, "Shutting it all down...\n- Hey, don't turn it off!"]]
Analyze [[49.345, 51.226, "I've got t